In [1]:
/home/phanim/harshitrawat/summer/T1_T2_T3_data/mp_traj_combinedxyz

NameError: name 'home' is not defined

In [9]:
# Now universal MACE finetuning on T2
#!/usr/bin/env python3
import os
import subprocess
import sys

def main():
    # ——— Environment setup ———
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    os.environ["PYTHONPATH"] = "/home/phanim/harshitrawat/mace/mace"

    cmd = [
        "torchrun",
        "--standalone", 
        "--nnodes=1", 
        "--nproc_per_node=1", 
        "/home/phanim/harshitrawat/mace/mace/cli/run_train.py",
        "--name",              "mace_T2_mp_replay_avg",
        "--model",             "MACE",
        "--num_interactions",  "2",
        "--foundation_model",  "/home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model",
        "--foundation_model_readout",
        "--pt_train_file","/home/phanim/harshitrawat/summer/T1_T2_T3_data/mp_traj_LiOZrLa_10k.extxyz",              # <- MP replay shortcut
        "--atomic_numbers","[3,8,40,57]",    # Li, O, Zr, La
        "--multiheads_finetuning","True",

        "--train_file","/home/phanim/harshitrawat/summer/T1_T2_T3_data/T2_it_2.extxyz",
        "--valid_file","/home/phanim/harshitrawat/summer/T1_T2_T3_data/val_it2.extxyz",


        "--batch_size",        "2",
        "--valid_batch_size",  "1",

        "--device",            "cuda",

        # === Loss function weights ===
        "--forces_weight",     "40",         # Increased force weight to balance energy better
        "--energy_weight",     "10",         # Reduced from 100 → avoid dominance + stabilize energy RMSE

        # === Learning setup ===
        "--lr",                "0.001",      # Explicit learning rate (0.0001 is too low → stagnation)
        "--scheduler_patience","4",          # Reduce LR if val loss doesn’t improve in 3 epochs
        "--clip_grad",         "2",        # Avoid exploding gradients — essential when energy_weight is high
        "--weight_decay",      "1e-8",       # Mild regularization to prevent overfitting

        # === EMA helps smooth loss curve ===
        "--ema_decay",         "0.999",     # Smooths validation loss and helps final convergence

        # === Domain + training settings ===
        "--r_max",             "5.0",
        "--max_num_epochs",    "130",
        "--E0s",               "average",    # Still allowed — could optionally be replaced by manual E0s
        "--seed",              "42",
        "--patience",     "8",

        "--restart_latest",                   # Resumes from checkpoint if available
    ]

    print("Running:", " \\\n    ".join(cmd), file=sys.stderr)
    subprocess.run(cmd, check=True)

if __name__ == "__main__":
    main()


Running: torchrun \
    --standalone \
    --nnodes=1 \
    --nproc_per_node=1 \
    /home/phanim/harshitrawat/mace/mace/cli/run_train.py \
    --name \
    mace_T2_mp_replay_avg \
    --model \
    MACE \
    --num_interactions \
    2 \
    --foundation_model \
    /home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model \
    --foundation_model_readout \
    --pt_train_file \
    /home/phanim/harshitrawat/summer/T1_T2_T3_data/mp_traj_LiOZrLa_10k.extxyz \
    --atomic_numbers \
    [3,8,40,57] \
    --multiheads_finetuning \
    True \
    --train_file \
    /home/phanim/harshitrawat/summer/T1_T2_T3_data/T2_it_2.extxyz \
    --valid_file \
    /home/phanim/harshitrawat/summer/T1_T2_T3_data/val_it2.extxyz \
    --batch_size \
    2 \
    --valid_batch_size \
    1 \
    --device \
    cuda \
    --forces_weight \
    40 \
    --energy_weight \
    10 \
    --lr \
    0.001 \
    --scheduler_patience \
    4 \
    --clip_grad \
    2 \
    --weight_

2025-08-18 10:07:07.114 INFO: ===========VERIFYING SETTINGS===========
2025-08-18 10:07:07.115 INFO: MACE version: 0.3.14
2025-08-18 10:07:07.704 INFO: CUDA version: 12.6, CUDA device: 0


/home/phanim/harshitrawat/mace/mace/cli/run_train.py:146: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  model_foundation = torch.load(


2025-08-18 10:07:08.189 INFO: Using foundation model /home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model as initial checkpoint.


Traceback (most recent call last):
  File "/home/phanim/harshitrawat/mace/mace/cli/run_train.py", line 996, in <module>
    main()
    ~~~~^^
  File "/home/phanim/harshitrawat/mace/mace/cli/run_train.py", line 77, in main
    run(args)
    ~~~^^^^^^
  File "/home/phanim/harshitrawat/mace/mace/cli/run_train.py", line 166, in run
    args.E0s != "average"
AssertionError: average atomic energies cannot be used for multiheads finetuning
E0818 10:07:09.017000 1098002 site-packages/torch/distributed/elastic/multiprocessing/api.py:874] failed (exitcode: 1) local_rank: 0 (pid: 1098035) of binary: /home/phanim/harshitrawat/miniconda3/bin/python
Traceback (most recent call last):
  File "/home/phanim/harshitrawat/miniconda3/bin/torchrun", line 8, in <module>
    sys.exit(main())
             ~~~~^^
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 355, in wrapper
    return f(*args, **kwargs)
  File "/hom

CalledProcessError: Command '['torchrun', '--standalone', '--nnodes=1', '--nproc_per_node=1', '/home/phanim/harshitrawat/mace/mace/cli/run_train.py', '--name', 'mace_T2_mp_replay_avg', '--model', 'MACE', '--num_interactions', '2', '--foundation_model', '/home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model', '--foundation_model_readout', '--pt_train_file', '/home/phanim/harshitrawat/summer/T1_T2_T3_data/mp_traj_LiOZrLa_10k.extxyz', '--atomic_numbers', '[3,8,40,57]', '--multiheads_finetuning', 'True', '--train_file', '/home/phanim/harshitrawat/summer/T1_T2_T3_data/T2_it_2.extxyz', '--valid_file', '/home/phanim/harshitrawat/summer/T1_T2_T3_data/val_it2.extxyz', '--batch_size', '2', '--valid_batch_size', '1', '--device', 'cuda', '--forces_weight', '40', '--energy_weight', '10', '--lr', '0.001', '--scheduler_patience', '4', '--clip_grad', '2', '--weight_decay', '1e-8', '--ema_decay', '0.999', '--r_max', '5.0', '--max_num_epochs', '130', '--E0s', 'ave

In [6]:
from ase.io import iread, write
import random

# Input / Output
in_path  = "/home/phanim/harshitrawat/summer/T1_T2_T3_data/mp_traj_combinedxyz"
out_path = "/home/phanim/harshitrawat/summer/T1_T2_T3_data/mp_traj_LiOZrLa_10k.extxyz"

ALLOWED = {3, 8, 40, 57}  # Li, O, Zr, La
K = 10000                 # target subsample size

reservoir = []
kept = 0
i = -1

# Reservoir sampling over stream
for i, at in enumerate(iread(in_path, ":")):
    zs = {a.number for a in at}
    if not zs.issubset(ALLOWED):
        continue
    kept += 1
    if len(reservoir) < K:
        reservoir.append(at)
    else:
        j = random.randint(0, kept - 1)
        if j < K:
            reservoir[j] = at

print(f"scanned={i+1}, kept_allowed={kept}, written={len(reservoir)} -> {out_path}")
write(out_path, reservoir)


scanned=145923, kept_allowed=142, written=142 -> /home/phanim/harshitrawat/summer/T1_T2_T3_data/mp_traj_LiOZrLa_10k.extxyz


In [5]:
from ase.io import iread
from collections import Counter

def list_unique_elements(path: str):
    Zcnt = Counter()
    frames = 0
    for at in iread(path, ":"):
        Zcnt.update(a.number for a in at)
        frames += 1
    print("File:", path)
    print("Frames:", frames)
    print("Unique Z:", sorted(Zcnt))
    print("Counts per Z:", dict(Zcnt))

# Example: scan the replay file before filtering
list_unique_elements(REPLAY_IN)


File: /home/phanim/harshitrawat/summer/T1_T2_T3_data/mp_traj_combinedxyz
Frames: 145923
Unique Z: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 89, 90, 91, 92, 93, 94]
Counts per Z: {83: 23906, 74: 18002, 8: 1520796, 20: 36708, 78: 9371, 7: 110283, 3: 127038, 12: 108165, 31: 26490, 73: 12543, 27: 50278, 16: 127700, 11: 48631, 14: 76839, 5: 59881, 25: 61445, 9: 156375, 38: 31429, 24: 27749, 68: 9326, 30: 31255, 19: 37916, 62: 11413, 15: 112799, 58: 12640, 1: 225974, 29: 38990, 52: 27776, 79: 8836, 34: 58621, 17: 67895, 48: 14870, 39: 16657, 66: 9056, 47: 16178, 65: 9224, 57: 23620, 67: 8927, 41: 20056, 56: 35329, 55: 16296, 32: 30773, 71: 6263, 75: 6755, 51: 26709, 35: 28462, 76: 4547, 63: 7071, 40: 15436, 61: 116